## 调库和准备

In [18]:
import os

import pandas as pd

import cv2
import numpy as np

from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, Model
from keras.layers import Embedding, LSTM, Input, Flatten, Dense,Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences

from PIL import Image
from progressbar import ProgressBar
from tqdm import tqdm
import time
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

pbar = ProgressBar()

tags_len = 30
pos_images = []
neg_images = []
pos_labels = []
neg_labels = []
y = []
features = []
light_dict = {}
pos_dir = ""
neg_dir = ""
# 读取图像和标签数据
pos_dir = "C:/Users/younger/Pictures/ApplePic/photo_tag/useful_1"
neg_dir = "C:/Users/younger/Pictures/ApplePic/photo_tag/useless_1"

kernel1 = ([-1,-1,-1],
           [-1, 8,-1],
           [-1,-1,-1])

## 图片导入预处理

In [19]:
kernel_conv = np.array(kernel1)

print("正在获取并预处理照片...\n")
print("positive images:\n")

for f in tqdm(os.listdir(pos_dir)):
    if f.endswith(".txt"):
        with open(os.path.join(pos_dir, f)) as file:
            lines = file.read().splitlines()
            for line in lines:
                line = line.split(',')
                pos_labels.append(line)
    elif f.endswith(".png"):
        img_lines = cv2.imread(os.path.join(pos_dir, f))
        #conv_img = cv2.filter2D(img_lines, cv2.CV_64F, kernel_conv)
        pos_images.append(img_lines)

print("negative images:\n")
for f in tqdm(os.listdir(neg_dir)):
    if f.endswith(".txt"):
        with open(os.path.join(neg_dir, f)) as file:
            lines = file.read().splitlines()
            for line in lines:
                line = line.split(',')
                neg_labels.append(lines)
    if f.endswith(".png"):
        img_lines = cv2.imread(os.path.join(neg_dir, f))
        neg_images.append(img_lines)



正在获取并预处理照片...

positive images:



100%|██████████| 14/14 [00:00<00:00, 297.88it/s]


negative images:



100%|██████████| 14/14 [00:00<00:00, 311.11it/s]


## 构建特征向量矩阵

In [20]:
def extract_image_features(image):
    # 加载 VGG16 模型
    model = VGG16(weights='imagenet', include_top=False)
    # 预处理图像
    img = cv2.resize(image, (512,512))
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    #print(img)
    # 提取特征向量
    img_features = model.predict(img, verbose=0)
    img_features = img_features.flatten()
    return img_features

# 提取标签特征向量
def extract_tag_features(labels):
    # 构建标签向量
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(labels)
    sequences = tokenizer.texts_to_sequences(labels)
    tag_features = pad_sequences(sequences, maxlen=tags_len)
    # 构建 LSTM 模型
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64, input_length=tags_len))
    model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
    tag_features = model.predict(tag_features, verbose=0)
    tag_features = np.pad(tag_features, ((0, tags_len - len(tag_features)), (0, 0)), mode='constant')
    print(tag_features.shape)
    return tag_features


## 提取特征向量

In [21]:

scaler = StandardScaler()
print("正在提取特征向量...\n")
y = []
features = []
for img, lables in tqdm(zip(pos_images, pos_labels)):
  img_features = extract_image_features(img)
  tag_features = extract_tag_features(lables)
  signle_features = np.concatenate(
    (tag_features, np.tile(img_features, (len(tag_features), 1))), axis=1
  )
  # 标准化特征矩阵
  signle_features = scaler.fit_transform(signle_features)
  y.append(1)
  features.append(signle_features)

for img, lables in tqdm(zip(neg_images, neg_labels)):
  img_features = extract_image_features(img)
  tag_features = extract_tag_features(lables)
  signle_features = np.concatenate(
    (tag_features, np.tile(img_features, (len(tag_features), 1))), axis=1
  )
  # 标准化特征矩阵
  signle_features = scaler.fit_transform(signle_features)
  y.append(0)
  features.append(signle_features)
print("特征提取完毕！\n")


正在提取特征向量...



1it [00:01,  1.18s/it]

(30, 64)


2it [00:02,  1.17s/it]

(30, 64)


...

(30, 64)


6it [00:07,  1.21s/it]

(30, 64)


7it [00:08,  1.21s/it]

(30, 64)
特征提取完毕！



## SVM

In [22]:
def train_model_SVM(X_train, X_test, y_train, y_test):
    accuracy = []
    precision = []
    recall = []
    f1 = []
    kernel_list = ["poly", "rbf", "sigmoid"]
    C_list = [0.1, 1, 10, 100]
    print("kernel\tC\taccuracy\tprecision\trecall\tf1")
    for kernel in tqdm(kernel_list):
        for C in tqdm(C_list):
            print("正在训练模型...\n")
            print("kernel=", kernel, "C=", C)
            clf = SVC(kernel=kernel, C=C)
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            accuracy = accuracy.append(accuracy_score(y_test, y_pred))
            precision = precision.append(
                precision_score(y_test, y_pred, average="macro")
            )
            recall = recall.append(recall_score(y_test, y_pred, average="macro"))
            f1 = f1.append(f1_score(y_test, y_pred, average="macro"))
            time.sleep(0.01)

    data = {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}
    df = pd.DataFrame(data, index=pd.MultiIndex.from_product([kernel_list, C_list], names=['kernel', 'C']))

    # 打印表格
    print(df)



## 划分数据集

In [23]:
features_1d=[feature.flatten() for feature in features]

In [24]:

# 将数据划分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(features_1d, y, test_size=0.2, random_state=42)


## CNN

n_matrices = len(X_train)
n_rows, n_cols = X_train[0].shape

# 将二维矩阵列表features转换为三维矩阵X
X_train = np.array(X_train).reshape((n_matrices, n_rows, n_cols,1))
y_train = np.array(y_train)


# 定义模型的输入层
input_layer = Input(shape=X_train.shape[1:])

# 定义模型的卷积层和池化层
conv1 = Conv2D(32, kernel_size=(3, 3), activation='relu')(input_layer)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(64, kernel_size=(3, 3), activation='relu')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

# 定义模型的全连接层和输出层
flat1 = Flatten()(pool2)
dense1 = Dense(64, activation='relu')(flat1)
output_layer = Dense(1, activation='sigmoid')(dense1)

# 定义模型并编译
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 训练模型
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


In [25]:
print(np.array(y).shape)

(14,)


In [26]:
import numpy as np

# 假设您的一维列表data已经准备好
n_sources = 3
n_samples = 10
n_features = 5
data = np.array([1, 1, 0] * n_samples * n_features)  # repeat the array to match the desired shape
print(data.shape)
print(data)
data = data.reshape((n_sources, n_samples, n_features))  # reshape the array to the desired shape
# 将一维列表data转换为三维矩阵X
print(data.shape)
print(data)

(150,)
[1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1
 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1
 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0
 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1
 1 0]
(3, 10, 5)
[[[1 1 0 1 1]
  [0 1 1 0 1]
  [1 0 1 1 0]
  [1 1 0 1 1]
  [0 1 1 0 1]
  [1 0 1 1 0]
  [1 1 0 1 1]
  [0 1 1 0 1]
  [1 0 1 1 0]
  [1 1 0 1 1]]

 [[0 1 1 0 1]
  [1 0 1 1 0]
  [1 1 0 1 1]
  [0 1 1 0 1]
  [1 0 1 1 0]
  [1 1 0 1 1]
  [0 1 1 0 1]
  [1 0 1 1 0]
  [1 1 0 1 1]
  [0 1 1 0 1]]

 [[1 0 1 1 0]
  [1 1 0 1 1]
  [0 1 1 0 1]
  [1 0 1 1 0]
  [1 1 0 1 1]
  [0 1 1 0 1]
  [1 0 1 1 0]
  [1 1 0 1 1]
  [0 1 1 0 1]
  [1 0 1 1 0]]]


In [30]:
# 训练模型
accuracy = pd.DataFrame(columns=['accuracy'])
precision = pd.DataFrame(columns=['precision'])
recall = pd.DataFrame(columns=['recall'])
f1 = pd.DataFrame(columns=['f1'])
kernel_list = ["poly", "rbf", "sigmoid"]
C_list = [0.1, 1, 10, 100]
print("kernel\tC\taccuracy\tprecision\trecall\tf1")
for kernel in tqdm(kernel_list):
    for C in tqdm(C_list):
        print("正在训练模型...\n")
        print("kernel=", kernel, "C=", C)
        clf = SVC(kernel=kernel, C=C)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        accuracy = accuracy.append({'accuracy': accuracy_score(y_test, y_pred)}, ignore_index=True)
        precision = precision.append({'precision': precision_score(y_test, y_pred)}, ignore_index=True)
        recall = recall.append({'recall': recall_score(y_test, y_pred)}, ignore_index=True)
        f1 = f1.append({'f1': f1_score(y_test, y_pred)}, ignore_index=True)

# 保存表格
from itertools import product
df = pd.concat([accuracy, precision, recall, f1], axis=1)
df.columns = ['accuracy', 'precision', 'recall', 'f1']
df.index = [(kernel, C) for kernel, C in product(kernel_list, C_list)]

# 打印表格
print(df)




kernel	C	accuracy	precision	recall	f1


  0%|          | 0/3 [00:00<?, ?it/s]

正在训练模型...

kernel= poly C= 0.1


C:\Users\younger\AppData\Local\Temp\ipykernel_12812\3691135916.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  accuracy = accuracy.append({'accuracy': accuracy_score(y_test, y_pred)}, ignore_index=True)
C:\Users\younger\AppData\Local\Temp\ipykernel_12812\3691135916.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  precision = precision.append({'precision': precision_score(y_test, y_pred)}, ignore_index=True)
C:\Users\younger\AppData\Local\Temp\ipykernel_12812\3691135916.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recall = recall.append({'recall': recall_score(y_test, y_pred)}, ignore_index=True)
C:\Users\younger\AppData\Local\Temp\ipykernel_12812\3691135916.py:19: FutureWarning: The frame.append method is d

正在训练模型...

kernel= poly C= 1


...

正在训练模型...

kernel= sigmoid C= 10


C:\Users\younger\AppData\Local\Temp\ipykernel_12812\3691135916.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  accuracy = accuracy.append({'accuracy': accuracy_score(y_test, y_pred)}, ignore_index=True)
C:\Users\younger\AppData\Local\Temp\ipykernel_12812\3691135916.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  precision = precision.append({'precision': precision_score(y_test, y_pred)}, ignore_index=True)
C:\Users\younger\AppData\Local\Temp\ipykernel_12812\3691135916.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recall = recall.append({'recall': recall_score(y_test, y_pred)}, ignore_index=True)
C:\Users\younger\AppData\Local\Temp\ipykernel_12812\3691135916.py:19: FutureWarning: The frame.append method is d

正在训练模型...

kernel= sigmoid C= 100


C:\Users\younger\AppData\Local\Temp\ipykernel_12812\3691135916.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  accuracy = accuracy.append({'accuracy': accuracy_score(y_test, y_pred)}, ignore_index=True)
C:\Users\younger\AppData\Local\Temp\ipykernel_12812\3691135916.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  precision = precision.append({'precision': precision_score(y_test, y_pred)}, ignore_index=True)
C:\Users\younger\AppData\Local\Temp\ipykernel_12812\3691135916.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recall = recall.append({'recall': recall_score(y_test, y_pred)}, ignore_index=True)
C:\Users\younger\AppData\Local\Temp\ipykernel_12812\3691135916.py:19: FutureWarning: The frame.append method is d

                accuracy  precision  recall        f1
(poly, 0.1)     0.333333   0.333333     1.0  0.500000
(poly, 1)       0.333333   0.333333     1.0  0.500000
(poly, 10)      0.333333   0.333333     1.0  0.500000
(poly, 100)     0.333333   0.333333     1.0  0.500000
(rbf, 0.1)      0.333333   0.333333     1.0  0.500000
(rbf, 1)        0.333333   0.333333     1.0  0.500000
(rbf, 10)       0.666667   0.500000     1.0  0.666667
(rbf, 100)      0.666667   0.500000     1.0  0.666667
(sigmoid, 0.1)  0.333333   0.333333     1.0  0.500000
(sigmoid, 1)    0.333333   0.333333     1.0  0.500000
(sigmoid, 10)   0.666667   0.000000     0.0  0.000000
(sigmoid, 100)  0.666667   0.000000     0.0  0.000000
